# E21 프로젝트: 한국어 데이터로 챗봇 만들기

## 데이터 수집

[송영숙님 챗봇 데이터 활용](https://github.com/songys/Chatbot_data/blob/master/ChatbotData%20.csv)

## 데이터 전처리하기
영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만, 전체적으로 다른 전처리를 수행해야 할 수도 있다!

## SubwordTextEncoder 사용하기
한국어는 형태소 분석기를 사용해서 토크나이징을 해야한다고 많이들 이야기 하지만, 실습에서 사용한 SubwordTextEncoder를 사용해보자.

## 모델 구성하기

## 모델 평가하기
랜덤하게 10개정도 추출해보고 판단해보자.